In [20]:
from chinese_checkers.simulation import SimulationCatalog, GameSimulation, SimulationMetadata

from typing import List

import pandas as pd
from collections import Counter

In [17]:
# filter to only 2 player games on a size 4 board
catalog = SimulationCatalog()
metadata_list: List[SimulationMetadata] = [
    m
    for m in catalog.list_available_metadata()
    if m.player_count == 2 and m.board_size == 4
]

for meta in metadata_list:
    print(meta)

SimulationMetadata(player_count=2, board_size=4, max_game_length=1000, winning_player='0', name='bootstrap-simulation', version='v0.0.1')
SimulationMetadata(player_count=2, board_size=4, max_game_length=1000, winning_player='3', name='bootstrap-simulation', version='v0.0.1')
SimulationMetadata(player_count=2, board_size=4, max_game_length=1000, winning_player='0', name='bad-player-3-bootstrap-simulation', version='v0.0.1')
SimulationMetadata(player_count=2, board_size=4, max_game_length=1000, winning_player='3', name='bad-player-3-bootstrap-simulation', version='v0.0.1')


In [21]:
# load in the simulations
simulations: List[GameSimulation] = []
for metadata in metadata_list:
    simulations += list(catalog.load_simulations_by_metadata(metadata))
    print(f"Metadata {metadata}, Simulation Count {len(simulations)}")

Metadata SimulationMetadata(player_count=2, board_size=4, max_game_length=1000, winning_player='0', name='bootstrap-simulation', version='v0.0.1'), Simulation Count 870
Metadata SimulationMetadata(player_count=2, board_size=4, max_game_length=1000, winning_player='3', name='bootstrap-simulation', version='v0.0.1'), Simulation Count 1704
Metadata SimulationMetadata(player_count=2, board_size=4, max_game_length=1000, winning_player='0', name='bad-player-3-bootstrap-simulation', version='v0.0.1'), Simulation Count 2133
Metadata SimulationMetadata(player_count=2, board_size=4, max_game_length=1000, winning_player='3', name='bad-player-3-bootstrap-simulation', version='v0.0.1'), Simulation Count 2167


In [26]:
# compute the summary statistics for the simulations
data = []
for sim in simulations:
    metadata: SimulationMetadata = sim.metadata
    data.append({
        "name": metadata.name,
        "version": metadata.version,
        "winning_player": metadata.winning_player,
        "game_length": len(sim.data.historical_moves)
    })

simulations_df = pd.DataFrame(data)

summary_df = simulations_df.groupby(["name", "version", "winning_player"]).agg(
    game_count=("winning_player", "size"),
    avg_game_length=("game_length", "mean"),
    min_game_length=("game_length", "min"),
    max_game_length=("game_length", "max"),
).reset_index()

metadata_summary_df = simulations_df.groupby(["name", "version"]).agg(
    total_games=("winning_player", "size"),
    winning_player_counts=("winning_player", lambda x: Counter(x))
).reset_index()

full_summary_df = pd.merge(summary_df, metadata_summary_df, on=["name", "version"])
full_summary_df

,name,version,winning_player,game_count,avg_game_length,min_game_length,max_game_length,total_games,winning_player_counts
0,bad-player-3-bootstrap-simulation,v0.0.1,0,429,473.783217,159,999,463,"{'0': 429, '3': 34}"
1,bad-player-3-bootstrap-simulation,v0.0.1,3,34,561.823529,292,842,463,"{'0': 429, '3': 34}"
2,bootstrap-simulation,v0.0.1,0,870,322.347126,115,515,1704,"{'0': 870, '3': 834}"
3,bootstrap-simulation,v0.0.1,3,834,321.707434,136,522,1704,"{'0': 870, '3': 834}"
